Bhavesh Bhatt

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

# Imports

In [65]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof

In [66]:
print(tf.__version__)

2.18.0


# Helper Functions

In [67]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [68]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

# Importa Data

In [69]:
import  pandas as pd

df = pd.read_csv("dataset.csv", header=None)              # No header in your format
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float16
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string

In [70]:
dataset = dataset.reshape(dataset.shape[0], 13, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (583554, 13, 16, 1)
Labels shape: (583554,)
Input shape: (13, 16, 1)


In [71]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.2, random_state=42, stratify=labels_set)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Print the lengths of the training, validation, and testing sets.
print(f"Training set length: {len(x_train)}")
print(f"Validation set length: {len(x_val)}")
print(f"Testing set length: {len(x_test)}")

Training set length: 373474
Validation set length: 93369
Testing set length: 116711


In [72]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights.
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
dist_class_weight = dict(enumerate(class_weight))

In [73]:
print(f"Classes: {np.unique(labels_set)}")

Classes: ['Background_noise' 'Bus' 'Car' 'Motorcycle' 'Truck']


In [74]:
print(f"Class weights: {dist_class_weight}")

Class weights: {0: np.float64(2.303546536729785), 1: np.float64(0.8562873290458667), 2: np.float64(0.8566506869738744), 3: np.float64(0.9921999946866449), 4: np.float64(0.8177576334833208)}


In [75]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))
y_val = to_categorical(label_encoder.fit_transform(y_val))

## Build & Compile the model


In [76]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    InputLayer(shape=input_shape),  # 25 frames, 16 channels

    Conv2D(filters=32, kernel_size=5, activation="relu", padding='same'),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    MaxPooling2D(pool_size=2),

    # Feature Pooling (Combining Max & Average Pooling)
    Flatten(),

    # Fully Connected Layer
    Dense(32, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.4),  # Dropout slightly reduced for stability

    # Output Layer (Softmax for multi-class classification)
    Dense(units=len(np.unique(labels_set)))
])

In [77]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 13, 16, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 6, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 6, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 3, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │        24,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,101 (172.27 KB)

 Trainable params: 44,101 (172.27 KB)

 Non-trainable params: 0 (0.00 B)

In [78]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy']
)

In [79]:
model.fit(x_train, y_train, epochs=4, validation_data=(x_val, y_val), class_weight=dist_class_weight, batch_size=64)

Epoch 1/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - categorical_accuracy: 0.2656 - loss: 1.5736 - val_categorical_accuracy: 0.3043 - val_loss: 1.4627
Epoch 2/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - categorical_accuracy: 0.2751 - loss: 1.4258 - val_categorical_accuracy: 0.3050 - val_loss: 1.4225
Epoch 3/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - categorical_accuracy: 0.2846 - loss: 1.3818 - val_categorical_accuracy: 0.2715 - val_loss: 1.4109
Epoch 4/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - categorical_accuracy: 0.2826 - loss: 1.3791 - val_categorical_accuracy: 0.3032 - val_loss: 1.4109


In [80]:
KERAS_MODEL_NAME = "tf_model_mini.h5"

In [81]:
model.save(KERAS_MODEL_NAME)

In [82]:
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

File size: 0.543 Megabytes


In [83]:
keras_model_size = get_file_size(KERAS_MODEL_NAME)

In [84]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy is {}%'.format(round(100*test_acc, 2)))

3648/3648 - 14s - 4ms/step - categorical_accuracy: 0.3024 - loss: 1.4130

Test accuracy is 30.24%


# TF Lite Model

In [85]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [86]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpwbnj9o_3/assets


INFO:tensorflow:Assets written to: /tmp/tmpwbnj9o_3/assets


Saved artifact at '/tmp/tmpwbnj9o_3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13, 16, 1), dtype=tf.float32, name='keras_tensor_173')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  131954152830432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152821984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152830080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152819872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152825504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152823392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131953613146592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131954152831840: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742390617.116834   24339 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742390617.116857   24339 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-19 14:23:37.117118: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpwbnj9o_3
2025-03-19 14:23:37.117734: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-19 14:23:37.117748: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpwbnj9o_3
2025-03-19 14:23:37.122639: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-19 14:23:37.151295: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpwbnj9o_3
2025-03-19 14:23:37.162031: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 44922 microseconds.


In [87]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

179628

In [88]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 175.418 Kilobytes


In [89]:
tflite_file_size = get_file_size(TF_LITE_MODEL_FILE_NAME)

# Check Input Tensor Shape

In [90]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 13 16  1]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


In [91]:
# Load your existing model
model = tf.keras.models.load_model('tf_model_mini.h5')

# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.uint8]

# Ensure that the input and output types are uint8
def representative_dataset_gen():
    for i in range(len(x_test)):
        # Get sample input data as a numpy array in a method of your choosing
        yield [x_test[i]]

converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

# Save the quantized model
with open('pipeline_int8_model_data.tflite', 'wb') as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: /tmp/tmpjm_090re/assets


INFO:tensorflow:Assets written to: /tmp/tmpjm_090re/assets


Saved artifact at '/tmp/tmpjm_090re'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13, 16, 1), dtype=tf.float32, name='input_layer_14')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  131956956190624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956956195552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956955582480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956955583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956956005824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956956006528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956956016208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131956956014272: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742390617.916418   24339 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742390617.916432   24339 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-19 14:23:37.916617: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjm_090re
2025-03-19 14:23:37.917263: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-19 14:23:37.917277: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpjm_090re
2025-03-19 14:23:37.923365: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-19 14:23:37.953526: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpjm_090re
2025-03-19 14:23:37.963240: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 46625 microseconds.


In [92]:
# Load and test the quantized model
interpreter = tf.lite.Interpreter(model_path='pipeline_int8_model_data.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random data from x_test
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

# Test the TensorFlow Lite model on random data from x_test
input_data = x_test[np.random.choice(x_test.shape[0], 1)]
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print("TensorFlow Lite model output:", tflite_results)

TensorFlow Lite model output: [[-0.8596065   0.30370185  0.25181004  0.28621268  0.27734286]]


In [93]:
!xxd -i pipeline_int8_model_data.tflite > pipeline_int8_model_data.cpp